<a href="https://colab.research.google.com/github/sundarp17/semi-supervised/blob/main/train1_svc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd

In [3]:

train_1 = pd.read_csv(r'/content/train_data1.csv')
test = pd.read_csv(r'/content/test_data.csv')

In [5]:

train_1['Target'] = train_1['Target'].replace(['Others'],'Invalid')


In [6]:
train_1['Target'].unique()

array(['Invalid', 'Issue', 'Analysis', 'Facts', 'Conclusion',
       'Rule/Law/Holding'], dtype=object)

In [4]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)

train1_Xtfidf = tfidf_vect.fit_transform(train_1['Sentence'])


In [9]:
test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))

In [10]:
unlabel_1 = test.loc[:100]
print("length of unlabel_1",len(unlabel_1))
unlabel_2 = test.loc[101:200]
print("length of unlabel_2",len(unlabel_2))
unlabel_3 = test.loc[201:300]
print("length of unlabel_3",len(unlabel_3))
unlabel_4 = test.loc[301:400]
print("length of unlabel_4",len(unlabel_4))
unlabel_5 = test.loc[401:]
print("length of unlabel_5",len(unlabel_5))

length of unlabel_1 101
length of unlabel_2 100
length of unlabel_3 100
length of unlabel_4 100
length of unlabel_5 120


In [11]:
unlabel_2.reset_index(inplace=True)
del unlabel_2['index']
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
4,906,agreed accompany gave pawn ticket money reclai...,Facts


In [12]:
unlabel_3.reset_index(inplace=True)
del unlabel_3['index']
unlabel_3.head()

,Unnamed: 0,Sentence,Target
0,1882,find evidence amply sufficient sustain juryâs ...,Analysis
1,964,actions show thatâ,Invalid
2,1417,certification attending physician shown filed ...,Facts
3,2932,testified agent bland first left automobile wa...,Facts
4,1584,officer testified could smell odor alcohol app...,Facts


In [13]:
unlabel_4.reset_index(inplace=True)
del unlabel_4['index']
unlabel_4.head()

,Unnamed: 0,Sentence,Target
0,266,section 6 pl,Invalid
1,3469,amended answer appellant alleged bond invalid ...,Analysis
2,1471,evidence sufficient support conviction appella...,Analysis
3,1990,cases cited note 19,Invalid
4,2757,july 8 1960 order entered revoking said probat...,Facts


In [14]:
unlabel_5.reset_index(inplace=True)
del unlabel_5['index']
unlabel_5.head()

,Unnamed: 0,Sentence,Target
0,2979,overrule appellantâs contention court erred pe...,Analysis
1,2135,state regarded authority case,Invalid
2,1591,testimony adduced appellant mother lillie mae ...,Facts
3,658,appellant needle marks inside left elbow,Facts
4,2562,analysis contents 3 capsules chemist kenneth a...,Facts


In [15]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.calibration import CalibratedClassifierCV


X_train, x_val, Y_train, y_val = train_test_split(train1_Xtfidf,train_1['Target'],test_size=0.26,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model=calibrator.fit(X_train, Y_train)
pred_svm = model.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6780973451327433
                  precision    recall  f1-score   support

        Analysis       0.50      0.39      0.44       126
      Conclusion       0.78      0.54      0.64        46
           Facts       0.74      0.91      0.82       485
         Invalid       0.51      0.42      0.46       132
           Issue       0.64      0.53      0.58        51
Rule/Law/Holding       0.52      0.20      0.29        64

        accuracy                           0.68       904
       macro avg       0.62      0.50      0.54       904
    weighted avg       0.65      0.68      0.65       904



In [16]:
x_un1 = tfidf_vect.transform(unlabel_1['Sentence'])
pred_unlabel_1 = model.predict_proba(x_un1)

In [17]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_1:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 1, 2, 2, 2, 2, 2, 2, 2, 3]
[0.9908752660180181, 0.9804916222553818, 0.9971329530271175, 0.9998986128370089, 0.9979816249062522, 0.9970770321859119, 0.9968937622382757, 0.9980026590115233, 0.9913560336034946, 0.9986776596678174, 0.9980626705604028, 0.9947887406153181, 0.9969088134436612, 0.9850498983055911, 0.9800029969964568, 0.9902120698638587, 0.9952288038139857, 0.9928546136442733, 0.9882652644028714, 0.9978302827568999, 0.999514173936761, 0.9994539272338783, 0.9902360833247698, 0.9861327547437754, 0.9861539987547384, 0.9990050773470832, 0.9927946830571446, 0.999249999757944, 0.9914847863141907, 0.9935343794499767, 0.9935334508616998, 0.9899994041670394, 0.9999034109336365, 0.9950099205104759, 0.9800251837568315, 0.9879344675702694, 0.995997515626005, 0.9925273192854436, 0.9989604816461718, 0.9994777996401792, 0.9992516487244734, 0.9970411104966705]
[0, 1, 4, 5, 11, 12, 17, 18, 21, 23

In [18]:
unlabel_1=unlabel_1.loc[[0, 1, 4, 5, 11, 12, 17, 18, 21, 23, 24, 29, 31, 33, 34, 35, 36, 37, 39, 40, 45, 48, 51, 52, 55, 56, 60, 62, 68, 69, 70, 72, 76, 78, 79, 84, 87, 89, 94, 95, 96, 98],:]
unlabel_1.head()

,Unnamed: 0,Sentence,Target
0,2173,darden sr testified âœi saw staggering â prett...,Facts
1,466,additional moral justification may due complai...,Analysis
4,79,eligible leaves renew contacts family,Facts
5,1955,got downstairs ponce went car wife left home,Facts
11,1604,police notified deceased found lying mrs moers...,Facts


In [19]:
frame_1 = [train_1,unlabel_1]
train_1_1 = pd.concat(frame_1)
len(train_1_1)

3518

In [20]:
x_train_1 = tfidf_vect.transform(train_1_1['Sentence'])
x_train_1.shape
X_train, x_val, Y_train, y_val = train_test_split(x_train_1,train_1_1['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_1=calibrator.fit(X_train, Y_train)
pred_svm = model_1.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6789772727272727
                  precision    recall  f1-score   support

        Analysis       0.45      0.43      0.44        97
      Conclusion       0.78      0.57      0.66        37
           Facts       0.75      0.89      0.82       359
         Invalid       0.56      0.52      0.54       112
           Issue       0.72      0.50      0.59        42
Rule/Law/Holding       0.64      0.28      0.39        57

        accuracy                           0.68       704
       macro avg       0.65      0.53      0.57       704
    weighted avg       0.67      0.68      0.66       704



In [21]:
x_un2 = tfidf_vect.transform(unlabel_2['Sentence'])
pred_unlabel_2 = model_1.predict_proba(x_un2)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_2:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 2, 1, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2]
[0.9942789991733849, 0.9989806693445494, 0.98753549628716, 0.9837867844003757, 0.9970024165783665, 0.9969657723280647, 0.9999465281796976, 0.9962035388785244, 0.9955627424509612, 0.9930663238820404, 0.9896175675493727, 0.9956131328610972, 0.9999551241761194, 0.9993980780317062, 0.9997914980141084, 0.9875647466746605, 0.9992165754219242, 0.9842052233783887, 0.9957750667206251, 0.9801205655081338, 0.9994771131187304, 0.9992580022871926, 0.999860875303391, 0.9989665891129582, 0.9996565741835046, 0.9948515873101921, 0.9969611357882037, 0.9892845792026569, 0.9970467774752316, 0.981430233268957, 0.9922186214598159, 0.9991932590443047, 0.9976557535533137, 0.9989331920215497, 0.9907827077040451]
[0, 2, 4, 6, 7, 8, 12, 21, 29, 31, 32, 36, 37, 39, 42, 48, 49, 54, 55, 58, 59, 63, 64, 69, 70, 71, 75, 79, 83, 85, 86, 88, 90, 91, 99]
35
35
35


In [22]:
unlabel_2 = unlabel_2.loc[[0,2, 4, 6, 7, 8, 12, 21, 29, 31,32, 34, 36, 37, 39, 42, 48, 49, 54, 55, 58, 59, 63, 64, 69, 70, 71, 75, 79, 83, 85, 86, 88, 90,91, 99 ],:]
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
2,2072,appellant testify offer evidence behalf,Facts
4,906,agreed accompany gave pawn ticket money reclai...,Facts
6,570,appellant performed work asked 7500 money ther...,Facts
7,3150,finding door facing street seen man walking pr...,Facts


In [23]:
frame_2 = [train_1_1,unlabel_2]
train1_2 = pd.concat(frame_2)
len(train1_2)

3554

In [24]:
x_train_2 = tfidf_vect.transform(train1_2['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_2,train1_2['Target'],test_size=0.27,random_state=2)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_2=calibrator.fit(X_train, Y_train)
pred_svm = model_2.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.628125
                  precision    recall  f1-score   support

        Analysis       0.41      0.37      0.39       142
      Conclusion       0.69      0.41      0.51        54
           Facts       0.69      0.90      0.78       467
         Invalid       0.51      0.41      0.45       158
           Issue       0.68      0.35      0.46        66
Rule/Law/Holding       0.75      0.29      0.42        73

        accuracy                           0.63       960
       macro avg       0.62      0.45      0.50       960
    weighted avg       0.62      0.63      0.60       960



In [25]:
x_un3 = tfidf_vect.transform(unlabel_3['Sentence'])
pred_unlabel_3 = model_2.predict_proba(x_un3)

pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_3:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 3, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2]
[0.9977462696715891, 0.9992862479105957, 0.9943918642000574, 0.9958397167210661, 0.986405600487724, 0.9997466210201387, 0.9996226869933362, 0.9997554529176679, 0.9998866064003014, 0.9872110121161577, 0.99777583795318, 0.9996935168492187, 0.9951696683667439, 0.9988142681755316, 0.9850700986646913, 0.9955935925203219, 0.999475258022451, 0.9909684924212779, 0.9996408857277402, 0.9811246042529177, 0.9965463699950778, 0.9963832928536414, 0.9953141504983445, 0.9984565766749217, 0.9966888046059936, 0.9939574265557377, 0.9913142973066579, 0.9991775714243301, 0.9974924145605653, 0.9991974436711228, 0.995288438540103, 0.9977282361763032, 0.9996778402269253, 0.9967385255017177, 0.9981093262176713, 0.9977421071499413, 0.9811291500018537, 0.9990578117678371]
[0, 2, 3, 4, 6, 7, 8, 12, 20, 24, 26, 27, 29, 30, 33, 35, 36, 40, 43, 45, 46, 49, 51, 52, 54, 56, 57, 63, 64, 65, 66, 67, 68, 70,

In [26]:
unlabel_3=unlabel_3.loc[[0, 2, 3, 4, 6,7, 8, 12, 20, 24, 26, 27, 29, 30, 33, 35, 36, 40, 43, 45, 46, 49, 51, 52, 54, 56, 57, 63, 64, 65, 66, 67, 68,70, 72,  75, 82, 89],:]
frame_3 = [train1_2,unlabel_3]
train1_3 = pd.concat(frame_3)

x_train_3 = tfidf_vect.transform(train1_3['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_3,train1_3['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_3=calibrator.fit(X_train, Y_train)
pred_svm = model_3.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6675938803894298
                  precision    recall  f1-score   support

        Analysis       0.47      0.43      0.45       103
      Conclusion       0.72      0.55      0.63        38
           Facts       0.75      0.90      0.82       383
         Invalid       0.45      0.38      0.41       104
           Issue       0.70      0.52      0.60        44
Rule/Law/Holding       0.44      0.17      0.25        47

        accuracy                           0.67       719
       macro avg       0.59      0.49      0.52       719
    weighted avg       0.64      0.67      0.65       719



In [27]:
x_un4 = tfidf_vect.transform(unlabel_4['Sentence'])
pred_unlabel_4 = model_3.predict_proba(x_un4)


pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_4:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[0.9985477376079472, 0.9929632198134898, 0.9991363887985794, 0.9999665100137434, 0.9995170647369094, 0.9835772203733801, 0.9996362822738118, 0.9972208770576249, 0.9978531044037724, 0.9972627598935215, 0.9966841349864938, 0.9995016119263939, 0.9957713933513588, 0.998254735640672, 0.9946362889503417, 0.9997033148717591, 0.9986440087426925, 0.9990798838429596, 0.9949998243896575, 0.9998049296535748, 0.9884383787862183, 0.9976585038780452, 0.9845057287029655, 0.9959045248369055, 0.9984835780209593, 0.9894239188469841, 0.9886175679887982, 0.9807346816552978, 0.9874699034657175, 0.9926361864214318, 0.9928377430526774, 0.9928054748236841, 0.9976054592173054, 0.9991653838176391, 0.9999568546558035, 0.9889750397480382, 0.9978396512955647, 0.9949824221851393, 0.9920183739028874, 0.9871392547776089, 0.9908002001268297, 0.9972732188233376, 0.9983568039745382]
[2, 6, 7, 

In [28]:
unlabel_4=unlabel_4.loc[[2, 6, 7, 10, 15, 17, 18, 21, 22, 27, 28, 29, 30, 31, 32, 34, 37, 38, 39, 40, 42, 49, 50, 55, 56, 58, 60, 61, 63, 64, 67, 69, 70, 71, 75, 82, 85, 87, 89, 90, 91, 93, 97 ],:]

In [30]:
frame_4 = [train1_3,unlabel_4]
train1_4 = pd.concat(frame_4)
len(train1_4)
x_train_4 = tfidf_vect.transform(train1_4['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_4,train1_4['Target'],test_size=0.27,random_state=42)
support = svm.LinearSVC()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_4=calibrator.fit(X_train, Y_train)
pred_svm = model_4.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6832993890020367
                  precision    recall  f1-score   support

        Analysis       0.50      0.40      0.44       145
      Conclusion       0.70      0.46      0.55        50
           Facts       0.78      0.90      0.84       525
         Invalid       0.49      0.50      0.49       142
           Issue       0.54      0.51      0.53        49
Rule/Law/Holding       0.56      0.34      0.42        71

        accuracy                           0.68       982
       macro avg       0.60      0.52      0.55       982
    weighted avg       0.67      0.68      0.67       982



In [31]:
x_un5 = tfidf_vect.transform(unlabel_5['Sentence'])

pred_unlabel_5 = model_4.predict_proba(x_un5)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_5:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 3, 2, 5, 3, 2, 2, 2, 2, 2, 2, 1, 1, 3, 2, 2, 2, 2, 2, 2, 5, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 3, 0]
[0.998985411210239, 0.9832277564101469, 0.9938872712783461, 0.9973056709472891, 0.984043380888774, 0.9929807694487258, 0.9967344376871732, 0.9983251034510595, 0.9986141626651803, 0.9855537361651836, 0.9869109740442455, 0.9957663964428407, 0.9994831933128289, 0.9895019748007035, 0.9925773001892788, 0.9987389481880322, 0.9993617706002427, 0.9985547550869266, 0.9933131522812124, 0.9980444264160884, 0.9947817260750249, 0.9913151794095958, 0.9959645279046901, 0.9921484690812383, 0.9958567483999585, 0.9882795059480004, 0.9999190376064097, 0.9990831318849654, 0.9957422177007818, 0.9991767067552048, 0.9983058409786398, 0.9891165004469786, 0.9842899920153285, 0.9886222399686168, 0.9866192267312783, 0.9899835093995152, 0.9901649045651651]
[3, 7, 8, 12, 18, 19, 22, 23, 26, 27, 29, 32, 36, 46, 48, 52, 53, 60, 63, 65, 73, 74, 75, 76, 77, 78, 82, 86, 93, 94, 95, 97, 100, 103, 104, 107, 110

In [32]:
unlabel_5 = unlabel_5.loc[[3, 7, 8, 12, 18, 19, 22, 23, 26, 27, 29, 32, 36, 46, 48, 52, 53, 60, 63, 65, 73, 74, 75, 76, 77, 78, 82, 86, 93, 94, 95, 97, 100, 103, 104, 107, 110],:]

In [33]:
frame_5 = [train1_4,unlabel_5]
train1_5 = pd.concat(frame_5)
len(train1_5)
x_train_5 = tfidf_vect.transform(train1_5['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_5,train1_5['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_5=calibrator.fit(X_train, Y_train)
pred_svm = model_5.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6884353741496598
                  precision    recall  f1-score   support

        Analysis       0.51      0.43      0.46       101
      Conclusion       0.77      0.50      0.61        34
           Facts       0.78      0.89      0.83       399
         Invalid       0.50      0.54      0.52       115
           Issue       0.60      0.34      0.44        35
Rule/Law/Holding       0.61      0.33      0.43        51

        accuracy                           0.69       735
       macro avg       0.63      0.51      0.55       735
    weighted avg       0.68      0.69      0.67       735



In [34]:
t_p = tfidf_vect.transform(test['Sentence'])
test_pred = model_5.predict(t_p)
print('Accuracy %s' % accuracy_score(test_pred,test['Target']))

Accuracy 0.6813819577735125


In [35]:
from sklearn.metrics import precision_recall_fscore_support
print(precision_recall_fscore_support(test['Target'],test_pred,average='macro'))

(0.6474778731487335, 0.5839042570243648, 0.602835850853905, None)
